In [ ]:
import pickle
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# LOAD
file = 'df_master_1992-2017'
f = open(file,'rb') 
df = pickle.load(f) 
f.close()

In [ ]:
df_match = df[df.match == 'match']

**Features to focus on:**  
* Budget
* Gross
* Genre (dummy variables)
* Country (US or UK - dummy variables)
* Runtime
* Metacritic score
* Total nominations
* Total wins
* Month

In [ ]:
df = df_match[['imdb_name', 'imdb_year', 'budget', 'gross', 'drama', 'comedy', 'action', 'crime', 'romance', 
                 'sport', 'biography', 'mystery', 'musical', 'thriller', 'adventure',
                 'sci-fi', 'family', 'history', 'USA', 'UK', 'runtime', 'metacritic', 
                   'total_noms', 'total_wins', 'month', 'winner', 'nominee', 'bp_nominee', 'oscar_name', 'oscar_year']]

In [ ]:
df.head()

**Convert runtime, budget, gross into integers**

In [ ]:
for i in ['budget', 'runtime', 'gross']:
    df[i] = df[i].replace(np.nan, '')

In [ ]:
df['test'] = df.runtime.map(lambda x: re.findall(r'[0-9]+', x))
df['runtime'] = df.test.apply(lambda x: int(x[0]) if len(x)>0 else np.nan)

In [ ]:
df['gross_new'] = df.gross.map(lambda x: ''.join(e for e in x if e.isdigit() or e == '.'))
df['gross_dollar'] = df.gross.apply(lambda x: 1 if '$' in x else 0)
df['gross_new'] = df.gross_new.map(lambda x: int(x) if len(x)>0 else np.nan)

In [ ]:
df['budget_new'] = df.budget.map(lambda x: ''.join(e for e in x if e.isdigit() or e == '.'))
df['budget_dollar'] = df.budget.apply(lambda x: 1 if '$' in x else 0)

In [ ]:
df['budget_currency'] = df.budget.map(lambda x: x[0] if len(x)>0 else '')
df.budget_currency.value_counts()

**NOTE**: Need to drop budget - too many missings

## Handle missing data

In [ ]:
df.isnull().sum()

In [ ]:
# Replace with zero
for i in ['total_noms', 'total_wins', 'winner']:
    df.loc[df[i].isnull() , i] = 0

In [ ]:
# Replace with mean
for i in ['metacritic', 'gross_new', 'runtime']:
    df[i].fillna(df[i].mean(), inplace = True)

# Create new variables

In [ ]:
# Convert month to season and create dummies
seasons = {"season": {"January": 'Winter', "February": 'Spring', "March": 'Spring',
                     "April": 'Spring', "May": 'Spring', "June": 'Summer', "July": 'Summer', 
                     "August": 'Summer', "September": 'Summer', "October": 'Winter', 
                     "November": 'Winter', "December": 'Winter' }}

df['season'] = df.month
df.replace(seasons, inplace=True)

df_season = pd.get_dummies(df['season'])
df = pd.concat([df, df_season], axis=1)

In [ ]:
# Standardize continuous variables by year
for i in ['gross_new', 'metacritic', 'total_wins', 'total_noms']:
    df[i + str('_norm')] = df.groupby('imdb_year')[i].transform(lambda x: (x - x.mean()) / x.std())

In [ ]:
plt.hist(df.gross_new_norm.dropna(), alpha=0.9, color='blue')

In [ ]:
# Log transform gross, wins, and noms
for i in ['gross_new_norm', 'total_wins_norm', 'total_noms_norm']:
    df['log_' + i] = df[i].map(lambda x: np.log(x + 1))

In [ ]:
plt.hist(df.log_gross_new_norm.dropna(), alpha=0.9, color='blue')

In [ ]:
plt.hist(df.log_total_wins_norm.dropna(), alpha=0.9, color='blue')

In [ ]:
plt.hist(df.log_total_noms_norm.dropna(), alpha=0.9, color='blue')

## Export df to be used in model

In [ ]:
df_model = df[['imdb_name', 'imdb_year', 'log_gross_new_norm',
               'drama', 'comedy', 'action', 'crime', 'romance', 'sport', 'biography', 'mystery', 'musical', 
               'thriller', 'adventure', 'sci-fi', 'family', 'history', 'USA', 'UK', 
               'runtime', 'metacritic_norm', 'log_total_noms_norm', 'log_total_wins_norm', 
               'winner', 'season', 'Winter', 'Summer', 'Spring', 'nominee', 'bp_nominee', 'oscar_name', 'oscar_year']]

In [ ]:
# DUMP
file = 'df_master_1992-2017_cleaned'
fileobj = open(file,'wb') 
pickle.dump(df_model,fileobj)

In [ ]:
df_model.to_csv('oscars_df_clean.csv', index=False, header=True)